In [1]:
# IMPORTS
import sys
import numpy as np
import pandas as pd
import yaml
from pathlib import Path
import glob
import os
from functools import partial

In [2]:
from bokeh.plotting import figure, show
from bokeh.layouts import gridplot, column
from bokeh.io import output_notebook
from bokeh.plotting import ColumnDataSource
from bokeh.models import DatetimeTickFormatter
output_notebook()

Loading BokehJS ...

In [3]:
def get_config(file) -> dict:
    """
    Read in config file and return it as a dictionary.

    :parameter
    ----------
    file - String
        Location of config file
    
    :returns
    --------
    config - dict
        Configuration file in dictionary form.
    """
    try:
        with open(file, 'r') as stream:
            config = yaml.safe_load(stream)
    
        return config
    except FileNotFoundError as e:
        print(f"File: could not be found. Error {e}")
        sys.exit(1)

In [4]:
config = get_config("config.yaml")
data_dir = config['data']

In [105]:
class Data:
    
#     def __init__(self, test):
#         pass
    
    def read_NO2(self, file, *, skiprows=7, sep=";"):
        df = pd.read_csv(file, skiprows=skiprows, sep=sep)
        return df
    
    @staticmethod
    def rename_columns(self, df, columns) -> pd.DataFrame:
        df.rename(columns = columns, inplace = True)
        return df
    
    def reformat_date_column(column, *, format_date = '%Y%m%d%H%M') -> pd.Series:
        try:
            column = column.str.replace(" ", "").str.replace(":","")
            column = pd.to_datetime(column.astype(str), format=format_date)
            return column
        except AttributeError as e:
            print(f"Date column: {column.name}, has already been reformatted. Error: {e}")
            return column

    @staticmethod
    def missing_value_filter(df: pd.DataFrame, p_missing: float) -> pd.DataFrame:
        """
        Keep only the columns that pass the maximum required % of missing values.

        :parameters
        -----------
        df - pd.DataFrame
            Data frame
        p_missing - float
            Maximum percentage missing values allowed for a column.

        :returns
        --------
        df - pd.DataFrame
            Filtered data frame based on % missing values.
        """
        df = df[df.columns[df.isnull().mean() < p_missing]]
        return df
    
    @staticmethod
    def interpolate_values(df, method = 'cubicspline', limit_direction = 'forward', *args, **kwargs,):
        df.interpolate(method = method, limit_direction = limit_direction, *args, **kwargs)
        return df
    
data = Data()

In [67]:
class MetaData:
    
    def __init__(self, file):
        self.file = file
        self.df = self.read_data(self.file)
        self.df = self.remove_data(self.df, "StationsCode", axis = 0)
        self.create_lat_long_col()

    def read_data(self, file, *, nrows=6, sep=";", encoding='unicode_escape') -> pd.DataFrame:
        df = pd.read_csv(file, nrows=nrows, sep=sep, encoding=encoding).iloc[:, 4:].T
        df = self.rename_columns(df)
        return df
        
    def rename_columns(self, df) -> pd.DataFrame:
        df.columns = df.iloc[0,:]
        return df
    
    def create_lat_long_col(self) -> None:
        # Seperate the latitude and longitude column and assign it to their own column
        latitude_longitude = self.df["Latitude,Longitude"].str.strip("()").str.split(",", n = 1, expand = True)
        self.remove_data(self.df, "Latitude,Longitude", axis = 1) # Remove old column
        
        # Insert the chunk number column into the dataframe
        self.df.insert(1, column = "Latitude", value = latitude_longitude.iloc[:, 0])

        # Insert the patient id column into the dataframe
        self.df.insert(2, column = "Longitude", value = latitude_longitude.iloc[:, 1])
    
    @staticmethod
    def remove_data(df, name, *, axis = 1) -> pd.DataFrame:
        df.drop(name, axis = axis, inplace = True)
        return df
        
# metadata_instance = MetaData(file_2020)

# <a id="/2">1. Load data</a> 

First, load in the data from the year 2020. Then, load in the data of the year 2021 by combine all the seperate files into one data frame. 

## <a id="/3">1.1 Year 2020</a> 

In [87]:
file_2020 = Path(data_dir) / "2020" / "2020_NO2.csv"
# df_2020 = pd.read_csv(file_2020, skiprows=7, sep=";")
df_2020 = data.read_NO2(file_2020, skiprows=7, sep=";")
df_2020.head()

,Component,Bep.periode,Eenheid,Begindatumtijd,Einddatumtijd,NL01485,NL01487,NL01488,NL01489,NL01491,...,NL49022,NL49546,NL49551,NL49553,NL49561,NL49564,NL49565,NL49701,NL49703,NL49704
0,NO2,uur,�g/m�,20200101 00:00,20200101 01:00,45.8,48.5,47.8,36.9,41.4,...,30.4,26.8,35.2,21.5,27.8,13.1,22.9,49.2,21.6,17.1
1,NO2,uur,�g/m�,20200101 01:00,20200101 02:00,32.3,55.8,45.1,43.4,47.4,...,24.6,33.7,16.4,16.0,24.3,24.0,30.2,54.3,21.2,26.6
2,NO2,uur,�g/m�,20200101 02:00,20200101 03:00,32.3,42.8,32.9,39.3,37.4,...,22.9,39.6,23.9,24.8,27.6,26.9,30.6,50.9,22.4,32.6
3,NO2,uur,�g/m�,20200101 03:00,20200101 04:00,25.4,40.3,32.1,26.4,37.1,...,20.4,31.1,22.9,22.7,29.5,28.5,27.9,38.8,22.3,28.4
4,NO2,uur,�g/m�,20200101 04:00,20200101 05:00,24.3,31.3,24.3,23.1,27.1,...,25.1,26.7,26.3,25.0,29.1,25.8,27.0,29.1,25.8,28.7


### Clean data

In [95]:
# Rename the date columns
df_2020 = data.rename_columns(df_2020, columns = {"Begindatumtijd": "date_start", "Einddatumtijd": "date_end"})

# Set the datatype of the date and time colum to datetime.
df_2020['date_start'] = data.reformat_date_column(df_2020['date_start'], format_date='%Y%m%d%H%M')
df_2020['date_end'] = data.reformat_date_column(df_2020['date_end'], format_date='%Y%m%d%H%M')


df_2020.head()

Date column: date_start, has already been reformatted. Error: Can only use .str accessor with string values!
Date column: date_end, has already been reformatted. Error: Can only use .str accessor with string values!


,Component,Bep.periode,Eenheid,date_start,date_end,NL01485,NL01487,NL01488,NL01489,NL01491,...,NL49022,NL49546,NL49551,NL49553,NL49561,NL49564,NL49565,NL49701,NL49703,NL49704
0,NO2,uur,�g/m�,2020-01-01 00:00:00,2020-01-01 01:00:00,45.8,48.5,47.8,36.9,41.4,...,30.4,26.8,35.2,21.5,27.8,13.1,22.9,49.2,21.6,17.1
1,NO2,uur,�g/m�,2020-01-01 01:00:00,2020-01-01 02:00:00,32.3,55.8,45.1,43.4,47.4,...,24.6,33.7,16.4,16.0,24.3,24.0,30.2,54.3,21.2,26.6
2,NO2,uur,�g/m�,2020-01-01 02:00:00,2020-01-01 03:00:00,32.3,42.8,32.9,39.3,37.4,...,22.9,39.6,23.9,24.8,27.6,26.9,30.6,50.9,22.4,32.6
3,NO2,uur,�g/m�,2020-01-01 03:00:00,2020-01-01 04:00:00,25.4,40.3,32.1,26.4,37.1,...,20.4,31.1,22.9,22.7,29.5,28.5,27.9,38.8,22.3,28.4
4,NO2,uur,�g/m�,2020-01-01 04:00:00,2020-01-01 05:00:00,24.3,31.3,24.3,23.1,27.1,...,25.1,26.7,26.3,25.0,29.1,25.8,27.0,29.1,25.8,28.7


### Missing values

In [106]:
p = 0.4
# Filter out all columns with to many missing values
df_2020 = data.missing_value_filter(df_2020, p)

# Interpolate all the NO2 values with a cubic spline.
df_2020.iloc[:,5:] = data.interpolate_values(df_2020.iloc[:,5:], method = 'cubicspline', limit_direction = 'forward')
df_2020

,Component,Bep.periode,Eenheid,date_start,date_end,NL01485,NL01487,NL01488,NL01489,NL01491,...,NL49022,NL49546,NL49551,NL49553,NL49561,NL49564,NL49565,NL49701,NL49703,NL49704
0,NO2,uur,�g/m�,2020-01-01 00:00:00,2020-01-01 01:00:00,45.8,48.5,47.8,36.9,41.4,...,30.4,26.8,35.2,21.5,27.8,13.1,22.9,49.2,21.6,17.1
1,NO2,uur,�g/m�,2020-01-01 01:00:00,2020-01-01 02:00:00,32.3,55.8,45.1,43.4,47.4,...,24.6,33.7,16.4,16.0,24.3,24.0,30.2,54.3,21.2,26.6
2,NO2,uur,�g/m�,2020-01-01 02:00:00,2020-01-01 03:00:00,32.3,42.8,32.9,39.3,37.4,...,22.9,39.6,23.9,24.8,27.6,26.9,30.6,50.9,22.4,32.6
3,NO2,uur,�g/m�,2020-01-01 03:00:00,2020-01-01 04:00:00,25.4,40.3,32.1,26.4,37.1,...,20.4,31.1,22.9,22.7,29.5,28.5,27.9,38.8,22.3,28.4
4,NO2,uur,�g/m�,2020-01-01 04:00:00,2020-01-01 05:00:00,24.3,31.3,24.3,23.1,27.1,...,25.1,26.7,26.3,25.0,29.1,25.8,27.0,29.1,25.8,28.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8779,NO2,uur,�g/m�,2020-12-31 19:00:00,2020-12-31 20:00:00,55.4,63.2,57.1,49.7,67.0,...,42.3,58.0,42.0,10.1,51.4,40.3,39.6,45.8,30.6,53.3
8780,NO2,uur,�g/m�,2020-12-31 20:00:00,2020-12-31 21:00:00,52.9,68.8,61.3,54.6,61.5,...,50.8,57.7,32.9,17.7,52.5,43.5,41.6,53.5,33.9,54.0
8781,NO2,uur,�g/m�,2020-12-31 21:00:00,2020-12-31 22:00:00,51.2,67.2,64.4,58.6,59.8,...,51.7,55.9,43.9,23.6,54.4,46.2,43.3,45.7,42.5,53.5
8782,NO2,uur,�g/m�,2020-12-31 22:00:00,2020-12-31 23:00:00,48.5,62.5,60.0,59.4,59.6,...,47.0,63.2,12.3,12.1,55.6,42.1,39.9,46.7,53.1,59.6


0

## Meta Data

In [68]:
meta_data = MetaData(file_2020)

df_meta_data = meta_data.df
df_meta_data

StationsCode,Stationsnaam,Latitude,Longitude,Stationsgebied,Stationstype,Meetprincipe,Meetopstelling
NL01485,Hoogvliet-Leemkuil,51.867411,4.355242,stad,achtergrond,Chemiluminescentie,Teledyne API 200E chemiluminescent Nox Analyser
NL01487,Rotterdam Zuid-Pleinweg,51.891147,4.48069,regionaal,verkeer,Chemiluminescentie,Teledyne API 200E chemiluminescent Nox Analyser
NL01488,Rotterdam Zuid-Zwartewaalstraat,51.893617,4.487528,stad,achtergrond,Chemiluminescentie,Teledyne API 200E chemiluminescent Nox Analyser
NL01489,Ridderkerk-Hogeweg,51.869431,4.580058,stad,verkeer,Chemiluminescentie,Teledyne API 200E chemiluminescent Nox Analyser
NL01491,Rotterdam-Oost Sidelinge A13,51.938472,4.430692,stad,verkeer,Chemiluminescentie,Teledyne API 200E chemiluminescent Nox Analyser
...,...,...,...,...,...,...,...
NL49564,Hoofddorp-Hoofdweg,52.327464,4.715008,onbekend,onbekend,Chemiluminescentie,Teledyne API 200E chemiluminescent Nox Analyser
NL49565,Oude Meer-Aalsmeerderdijk,52.279991,4.770773,regionaal,achtergrond,Chemiluminescentie,Teledyne API 200E chemiluminescent Nox Analyser
NL49701,Zaandam-Wagenschotpad,52.448011,4.816706,stad,achtergrond,Chemiluminescentie,Thermo model 42w NO/Nox analyser
NL49703,Amsterdam-Spaarnwoude,52.398437,4.728581,regionaal,achtergrond,Chemiluminescentie,Teledyne API 200E chemiluminescent Nox Analyser


# ----Old test version----

### Clean data

In [7]:
# Rename the date columns
df_2020.rename(columns = {"Begindatumtijd": "date_start", "Einddatumtijd": "date_end"}, inplace = True)

# Set the datatype of the date and time colum to datetime.
df_2020.date_start = df_2020.date_start.str.replace(" ", "").str.replace(":","")
df_2020.date_start = pd.to_datetime(df_2020.date_start.astype(str), format='%Y%m%d%H%M')

df

df_2020.date_end = df_2020.date_end.str.replace(" ", "").str.replace(":","")
df_2020.date_end = pd.to_datetime(df_2020.date_end.astype(str), format='%Y%m%d%H%M')

df_2020.head()

,Component,Bep.periode,Eenheid,date_start,date_end,NL01485,NL01487,NL01488,NL01489,NL01491,...,NL49022,NL49546,NL49551,NL49553,NL49561,NL49564,NL49565,NL49701,NL49703,NL49704
0,NO2,uur,�g/m�,2020-01-01 00:00:00,2020-01-01 01:00:00,45.8,48.5,47.8,36.9,41.4,...,30.4,26.8,35.2,21.5,27.8,13.1,22.9,49.2,21.6,17.1
1,NO2,uur,�g/m�,2020-01-01 01:00:00,2020-01-01 02:00:00,32.3,55.8,45.1,43.4,47.4,...,24.6,33.7,16.4,16.0,24.3,24.0,30.2,54.3,21.2,26.6
2,NO2,uur,�g/m�,2020-01-01 02:00:00,2020-01-01 03:00:00,32.3,42.8,32.9,39.3,37.4,...,22.9,39.6,23.9,24.8,27.6,26.9,30.6,50.9,22.4,32.6
3,NO2,uur,�g/m�,2020-01-01 03:00:00,2020-01-01 04:00:00,25.4,40.3,32.1,26.4,37.1,...,20.4,31.1,22.9,22.7,29.5,28.5,27.9,38.8,22.3,28.4
4,NO2,uur,�g/m�,2020-01-01 04:00:00,2020-01-01 05:00:00,24.3,31.3,24.3,23.1,27.1,...,25.1,26.7,26.3,25.0,29.1,25.8,27.0,29.1,25.8,28.7


### Missing data

In [8]:
# Check missing value

def missing_value_filter(df: pd.DataFrame, p_missing: float) -> pd.DataFrame:
    """
    Keep only the columns that pass the maximum required % of missing values.
    
    :parameters
    -----------
    df - pd.DataFrame
        Data frame
    p_missing - float
        Maximum percentage missing values allowed for a column.
        
    :returns
    --------
    df - pd.DataFrame
        Filtered data frame based on % missing values.
    """
    df = df[df.columns[df.isnull().mean() < p_missing]]
    return df

df_2020.iloc[:,5:].isnull().mean()

# df_2021[df_2021.columns[df_2021.isnull().mean() < 0.4]]
df_2020[df_2020.columns[df_2020.isnull().mean() < 0.4]]
# df_2020.columns[df_2020.isnull().mean() < 0.4]
p = 0.4
df_2020 = missing_value_filter(df_2020, p)
df_2020


,Component,Bep.periode,Eenheid,date_start,date_end,NL01485,NL01487,NL01488,NL01489,NL01491,...,NL49022,NL49546,NL49551,NL49553,NL49561,NL49564,NL49565,NL49701,NL49703,NL49704
0,NO2,uur,�g/m�,2020-01-01 00:00:00,2020-01-01 01:00:00,45.8,48.5,47.8,36.9,41.4,...,30.4,26.8,35.2,21.5,27.8,13.1,22.9,49.2,21.6,17.1
1,NO2,uur,�g/m�,2020-01-01 01:00:00,2020-01-01 02:00:00,32.3,55.8,45.1,43.4,47.4,...,24.6,33.7,16.4,16.0,24.3,24.0,30.2,54.3,21.2,26.6
2,NO2,uur,�g/m�,2020-01-01 02:00:00,2020-01-01 03:00:00,32.3,42.8,32.9,39.3,37.4,...,22.9,39.6,23.9,24.8,27.6,26.9,30.6,50.9,22.4,32.6
3,NO2,uur,�g/m�,2020-01-01 03:00:00,2020-01-01 04:00:00,25.4,40.3,32.1,26.4,37.1,...,20.4,31.1,22.9,22.7,29.5,28.5,27.9,38.8,22.3,28.4
4,NO2,uur,�g/m�,2020-01-01 04:00:00,2020-01-01 05:00:00,24.3,31.3,24.3,23.1,27.1,...,25.1,26.7,26.3,25.0,29.1,25.8,27.0,29.1,25.8,28.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8779,NO2,uur,�g/m�,2020-12-31 19:00:00,2020-12-31 20:00:00,55.4,63.2,57.1,49.7,67.0,...,42.3,58.0,42.0,10.1,51.4,40.3,39.6,45.8,30.6,53.3
8780,NO2,uur,�g/m�,2020-12-31 20:00:00,2020-12-31 21:00:00,52.9,68.8,61.3,54.6,61.5,...,50.8,57.7,32.9,17.7,52.5,43.5,41.6,53.5,33.9,54.0
8781,NO2,uur,�g/m�,2020-12-31 21:00:00,2020-12-31 22:00:00,51.2,67.2,64.4,58.6,59.8,...,51.7,55.9,43.9,23.6,54.4,46.2,43.3,45.7,42.5,53.5
8782,NO2,uur,�g/m�,2020-12-31 22:00:00,2020-12-31 23:00:00,48.5,62.5,60.0,59.4,59.6,...,47.0,63.2,12.3,12.1,55.6,42.1,39.9,46.7,53.1,59.6


### Interpolate missing data

In [9]:
# Interpolate all the NO2 values with a cubic spline.
df_2020.iloc[:,5:] = df_2020.iloc[:,5:].interpolate(method = 'cubicspline', limit_direction = 'forward')

# df_2020['interpolated'] = df_2020['no2'].interpolate(method = 'cubicspline', limit_direction = 'forward')
# tidy_2020.iloc[index_missing_values,:]
# df_2020.iloc[:,5:].isnull().mean()
df_2020.head()

,Component,Bep.periode,Eenheid,date_start,date_end,NL01485,NL01487,NL01488,NL01489,NL01491,...,NL49022,NL49546,NL49551,NL49553,NL49561,NL49564,NL49565,NL49701,NL49703,NL49704
0,NO2,uur,�g/m�,2020-01-01 00:00:00,2020-01-01 01:00:00,45.8,48.5,47.8,36.9,41.4,...,30.4,26.8,35.2,21.5,27.8,13.1,22.9,49.2,21.6,17.1
1,NO2,uur,�g/m�,2020-01-01 01:00:00,2020-01-01 02:00:00,32.3,55.8,45.1,43.4,47.4,...,24.6,33.7,16.4,16.0,24.3,24.0,30.2,54.3,21.2,26.6
2,NO2,uur,�g/m�,2020-01-01 02:00:00,2020-01-01 03:00:00,32.3,42.8,32.9,39.3,37.4,...,22.9,39.6,23.9,24.8,27.6,26.9,30.6,50.9,22.4,32.6
3,NO2,uur,�g/m�,2020-01-01 03:00:00,2020-01-01 04:00:00,25.4,40.3,32.1,26.4,37.1,...,20.4,31.1,22.9,22.7,29.5,28.5,27.9,38.8,22.3,28.4
4,NO2,uur,�g/m�,2020-01-01 04:00:00,2020-01-01 05:00:00,24.3,31.3,24.3,23.1,27.1,...,25.1,26.7,26.3,25.0,29.1,25.8,27.0,29.1,25.8,28.7


### Create tidy data

In [10]:
# Example link: https://cmdlinetips.com/2019/06/reshaping-dataframes-with-pandas-melt-and-wide_to_long/
tidy_2020 = df_2020.melt(id_vars=["date_start", "date_end"],
            value_vars=df_2020.columns[5:],
                        var_name = "site",
                        value_name = "no2")
tidy_2020.head()

,date_start,date_end,site,no2
0,2020-01-01 00:00:00,2020-01-01 01:00:00,NL01485,45.8
1,2020-01-01 01:00:00,2020-01-01 02:00:00,NL01485,32.3
2,2020-01-01 02:00:00,2020-01-01 03:00:00,NL01485,32.3
3,2020-01-01 03:00:00,2020-01-01 04:00:00,NL01485,25.4
4,2020-01-01 04:00:00,2020-01-01 05:00:00,NL01485,24.3


In [11]:
# There is some missing data.
tidy_2020.isnull().sum().sum()

0

In [12]:
# missing_values = tidy_2020[tidy_2020["no2"].isnull()]
# index_missing_values = missing_values.index.values
# missing_values
# index_missing_values

array([], dtype=int64)

### Interpolate missing data with a cubic spline

In [108]:
# tidy_2020['interpolated'] = tidy_2020['no2'].interpolate(method = 'cubicspline', limit_direction = 'forward')
# tidy_2020.iloc[index_missing_values,:]

,date_start,date_end,site,no2,interpolated
96,2020-01-05 00:00:00,2020-01-05 01:00:00,NL01485,NaN,47.978060
97,2020-01-05 01:00:00,2020-01-05 02:00:00,NL01485,NaN,35.617949
98,2020-01-05 02:00:00,2020-01-05 03:00:00,NL01485,NaN,24.728807
99,2020-01-05 03:00:00,2020-01-05 04:00:00,NL01485,NaN,16.619777
101,2020-01-05 05:00:00,2020-01-05 06:00:00,NL01485,NaN,13.364538
...,...,...,...,...,...
640379,2020-11-26 11:00:00,2020-11-26 12:00:00,NL49704,NaN,31.175801
640403,2020-11-27 11:00:00,2020-11-27 12:00:00,NL49704,NaN,30.644382
640551,2020-12-03 15:00:00,2020-12-03 16:00:00,NL49704,NaN,24.485992
640836,2020-12-15 12:00:00,2020-12-15 13:00:00,NL49704,NaN,43.722142


In [13]:
# Grab the mean NO2 value per day for each site
mean_no2_day = tidy_2020.groupby(['site', tidy_2020['date_start'].dt.date]).agg({'no2': ['mean']})
mean_no2_day.columns = ["no2_mean"]
mean_no2_day

no2_mean
site    date_start           
NL01485 2020-01-01  22.029167
        2020-01-02  18.970833
        2020-01-03  19.004167
        2020-01-04  39.550000
        2020-01-05  20.236882
...                       ...
NL49704 2020-12-27  18.854167
        2020-12-28  32.970833
        2020-12-29  36.295833
        2020-12-30  28.425000
        2020-12-31  38.525000

[26718 rows x 1 columns]

In [14]:
tidy_2020['date_start'].dt.date

0         2020-01-01
1         2020-01-01
2         2020-01-01
3         2020-01-01
4         2020-01-01
             ...    
641227    2020-12-31
641228    2020-12-31
641229    2020-12-31
641230    2020-12-31
641231    2020-12-31
Name: date_start, Length: 641232, dtype: object

In [16]:
one_site = tidy_2020[tidy_2020.site == "NL10641"] # NL10641 NL01485

In [18]:
# CODE YOUR SOLUTION HERE
p = figure(plot_width = 500, plot_height = 400, tools="pan, hover", x_axis_type='datetime')

#p.line(df_glucose['time'], df_glucose['interpolated'], legend_label = "Interpolated data")
p.scatter(one_site["date_start"], one_site["no2"], color = "red", marker = "asterisk", size = 7, legend_label = "Glucose values")
p.legend.location = "top_left"
p.legend.click_policy="hide"
show(p)

In [19]:
# link: slicing multiindex https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html#advanced-xs
# df.xs("one", level="second")
NL10641_mean = mean_no2_day.xs('NL10641',level='site')
NL10641_mean

,no2_mean
date_start,
2020-01-01,28.436667
2020-01-02,28.187917
2020-01-03,22.930833
2020-01-04,17.119583
2020-01-05,26.365833
...,...
2020-12-27,13.314167
2020-12-28,19.744583
2020-12-29,28.828333


In [71]:
# Maybe take the mean over all station

NL10641_mean.index = pd.to_datetime(NL10641_mean.index, format="%Y-%m-%d")
# NL10641_mean.loc[NL10641_mean.index.weekday.values == 0:]
monday_mean = NL10641_mean[NL10641_mean.index.weekday == 0]
monday_mean

,no2_mean
date_start,
2020-01-06,32.273810
2020-01-13,27.239583
2020-01-20,58.935833
2020-01-27,27.935000
2020-02-03,19.412083
2020-02-10,11.145833
2020-02-17,17.888333
2020-02-24,20.759500
2020-03-02,28.007500


In [110]:
one_day_10641 = tidy_2020[(tidy_2020.date_start.dt.day == 1) & (tidy_2020.date_start.dt.month == 1) & (tidy_2020.site == 'NL01485')]['no2']
one_day_10641.reset_index(drop=True, inplace=True) 
one_day_10641

0     45.8
1     32.3
2     32.3
3     25.4
4     24.3
5     22.5
6     22.5
7     25.2
8     22.2
9     22.9
10    13.9
11    10.9
12    13.6
13    14.5
14    15.9
15    18.1
16    24.8
17    27.9
18    31.3
19    18.9
20    11.8
21    14.1
22    16.6
23    21.0
Name: no2, dtype: float64

In [91]:
day_two_10641 = tidy_2020[(tidy_2020.date_start.dt.day == 2) & (tidy_2020.date_start.dt.month == 1) & (tidy_2020.site == 'NL10641')]['no2']
day_two_10641.reset_index(drop=True, inplace=True) 
day_two_10641

0     25.71
1     14.80
2     22.17
3     15.49
4     16.66
5     23.44
6     23.53
7     42.11
8     33.09
9     35.23
10    34.16
11    24.95
12    21.49
13    25.33
14    35.88
15    38.42
16    38.65
17    38.15
18    31.22
19    33.13
20    28.63
21    26.40
22    24.15
23    23.72
Name: no2, dtype: float64

In [98]:
# CODE YOUR SOLUTION HERE
p = figure(plot_width = 500, plot_height = 400, tools="pan, hover", x_axis_type='datetime')

# p.line(monday_mean.index, monday_mean["no2_mean"], legend_label = "Interpolated data")
p.line(one_day_10641.index, one_day_10641.values, legend_label = "Interpolated data")
# p.scatter(monday_mean.index, monday_mean["no2_mean"], color = "red", marker = "asterisk", size = 7, legend_label = "Glucose values")
p.legend.location = "top_left"
p.legend.click_policy="hide"
show(p)

In [92]:
p = figure(plot_width = 500, plot_height = 400, tools="pan, hover", x_axis_type='datetime')

# p.line(monday_mean.index, monday_mean["no2_mean"], legend_label = "Interpolated data")
p.line(day_two_10641.index, day_two_10641.values, legend_label = "Interpolated data")
# p.scatter(monday_mean.index, monday_mean["no2_mean"], color = "red", marker = "asterisk", size = 7, legend_label = "Glucose values")
p.legend.location = "top_left"
p.legend.click_policy="hide"
show(p)

In [66]:
# mean_no2_day = tidy_2020.groupby(['site', tidy_2020['date_start'].dt.date]).agg({'no2': ['mean']})
# mean_no2_day.columns = ["no2_mean"]
# mean_no2_day
test = tidy_2020.groupby(['site', tidy_2020['date_start'].dt.date]).agg({'no2': ['mean']})
test.reset_index(inplace=True, level = ['date_start'])
test[test.index == 'NL10641']

,date_start,no2
,,mean
site,,
NL10641,2020-01-01,28.436667
NL10641,2020-01-02,28.187917
NL10641,2020-01-03,22.930833
NL10641,2020-01-04,17.119583
NL10641,2020-01-05,26.365833
...,...,...
NL10641,2020-12-27,13.314167
NL10641,2020-12-28,19.744583


## Interpolation and taking average

"The daily data was interpolated using inverse distance weighted interpolation with a maximum distance of 0.05 degrees to fill in gaps between valid observations but avoiding extrapolation outside areas with valid observations. After interpolation, the daily observations were averaged over time to obtain June-August average NO2 levels." - https://www.greenpeace.org.au/research/new-satellite-data-reveals-worlds-largest-air-pollution-emission-hotspots-greenpeace-media-briefing/

In [ ]:
# Interpolate the missing data, and take the avarage NO2 value per day.

In [133]:
def read_csv_file(file, *, skiprows = 0, sep = ",", encoding = None) -> pd.DataFrame:
    """
    Read in a csv file.
    
    :parameters
    -----------
    file - String
        File location
    skiprows - int
        Number of rows to skip
    sep - str
        Seperator
    encoding - str
        Encoding to use for UTF when reading/writing (ex. ‘utf-8’).
        
    :return
    -------
    df - pd.DataFrame
        Pandas data frame
    """
    df = pd.read_csv(file, skiprows=7, sep=";", encoding='unicode_escape')
    return df
   
# Map does not take keyword arugments: solution create a partial.
map_func = partial(read_csv_file, skiprows=7, sep=";", encoding='unicode_escape')

# Merging the files 2021 csv files
list_2021 = glob.glob(os.path.join(data_dir, "2021", "*.csv"))
df_2021 = pd.concat(map(map_func, list_2021), ignore_index=True)
df_2021

,Component,Bep.periode,Eenheid,Begindatumtijd,Einddatumtijd,NL01485,NL01487,NL01488,NL01489,NL01491,...,NL49022,NL49546,NL49551,NL49553,NL49561,NL49564,NL49565,NL49701,NL49703,NL49704
0,NO2,uur,µg/m³,20210101 00:00,20210101 01:00,41.4,57.3,50.2,46.4,58.1,...,39.4,54.6,30.3,19.6,47.5,35.0,43.4,45.8,42.0,51.5
1,NO2,uur,µg/m³,20210101 01:00,20210101 02:00,41.8,51.4,47.8,43.0,56.2,...,36.0,53.4,39.7,17.9,49.2,33.6,44.4,40.2,43.3,48.9
2,NO2,uur,µg/m³,20210101 02:00,20210101 03:00,35.5,51.0,45.1,40.7,52.0,...,42.2,54.1,8.9,3.9,53.0,35.6,44.9,47.6,41.9,50.5
3,NO2,uur,µg/m³,20210101 03:00,20210101 04:00,20.4,52.9,49.9,45.3,54.6,...,41.2,55.1,9.0,3.0,56.6,38.9,56.0,42.7,39.3,47.1
4,NO2,uur,µg/m³,20210101 04:00,20210101 05:00,14.5,46.2,45.6,43.5,54.4,...,48.8,56.4,12.5,2.4,58.3,40.3,55.8,46.1,41.8,47.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8011,NO2,uur,µg/m³,20211130 19:00,20211130 20:00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8012,NO2,uur,µg/m³,20211130 20:00,20211130 21:00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8013,NO2,uur,µg/m³,20211130 21:00,20211130 22:00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8014,NO2,uur,µg/m³,20211130 22:00,20211130 23:00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [142]:
df_2021.isnull().sum().sum()

# df_2021.iloc[:,5:].isnull().mean()
# df_2021[df_2021.columns[df_2021.isnull().mean() < 0.1]]
# df_2021.columns[df_2021.isnull().mean() < 0.4]
df_2021[df_2021.columns[df_2021.isnull().mean() < 0.4]]

,Component,Bep.periode,Eenheid,Begindatumtijd,Einddatumtijd,NL01485,NL01487,NL01488,NL01489,NL01491,...,NL10738,NL10741,NL10742,NL10807,NL10818,NL10918,NL10929,NL10934,NL10937,NL10938
0,NO2,uur,µg/m³,20210101 00:00,20210101 01:00,41.4,57.3,50.2,46.4,58.1,...,14.26,41.78,32.58,16.12,29.11,22.84,13.27,12.43,34.03,30.30
1,NO2,uur,µg/m³,20210101 01:00,20210101 02:00,41.8,51.4,47.8,43.0,56.2,...,12.45,42.19,33.53,15.62,32.72,20.13,14.64,15.12,24.56,22.76
2,NO2,uur,µg/m³,20210101 02:00,20210101 03:00,35.5,51.0,45.1,40.7,52.0,...,10.50,41.26,33.26,20.28,30.52,16.96,14.31,18.14,21.07,20.20
3,NO2,uur,µg/m³,20210101 03:00,20210101 04:00,20.4,52.9,49.9,45.3,54.6,...,9.52,40.00,32.02,24.03,30.22,18.91,17.05,13.87,20.22,18.98
4,NO2,uur,µg/m³,20210101 04:00,20210101 05:00,14.5,46.2,45.6,43.5,54.4,...,9.26,39.56,31.15,24.89,28.20,19.43,21.04,11.29,21.10,16.14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8011,NO2,uur,µg/m³,20211130 19:00,20211130 20:00,NaN,NaN,NaN,NaN,NaN,...,17.87,24.45,28.44,4.90,7.32,4.97,3.47,4.35,29.99,11.30
8012,NO2,uur,µg/m³,20211130 20:00,20211130 21:00,NaN,NaN,NaN,NaN,NaN,...,15.57,23.51,28.75,6.93,6.52,3.33,5.33,8.37,28.89,12.02
8013,NO2,uur,µg/m³,20211130 21:00,20211130 22:00,NaN,NaN,NaN,NaN,NaN,...,10.47,18.70,24.28,8.56,6.61,15.93,6.43,7.35,17.58,10.20
8014,NO2,uur,µg/m³,20211130 22:00,20211130 23:00,NaN,NaN,NaN,NaN,NaN,...,9.80,17.22,24.54,13.63,11.27,14.04,6.10,5.42,15.49,7.29


## Meta data

In [23]:
meta_data = pd.read_csv(file_2020, nrows=6, sep=";", encoding='unicode_escape').iloc[:, 4:].T
# Set first row as column names
meta_data.columns = meta_data.iloc[0,:]

# Remove the first row
meta_data.drop("StationsCode", axis = 0, inplace = True)
# meta_data
meta_data

StationsCode,Stationsnaam,"Latitude,Longitude",Stationsgebied,Stationstype,Meetprincipe,Meetopstelling
NL01485,Hoogvliet-Leemkuil,"(51.867411,4.355242)",stad,achtergrond,Chemiluminescentie,Teledyne API 200E chemiluminescent Nox Analyser
NL01487,Rotterdam Zuid-Pleinweg,"(51.891147,4.48069)",regionaal,verkeer,Chemiluminescentie,Teledyne API 200E chemiluminescent Nox Analyser
NL01488,Rotterdam Zuid-Zwartewaalstraat,"(51.893617,4.487528)",stad,achtergrond,Chemiluminescentie,Teledyne API 200E chemiluminescent Nox Analyser
NL01489,Ridderkerk-Hogeweg,"(51.869431,4.580058)",stad,verkeer,Chemiluminescentie,Teledyne API 200E chemiluminescent Nox Analyser
NL01491,Rotterdam-Oost Sidelinge A13,"(51.938472,4.430692)",stad,verkeer,Chemiluminescentie,Teledyne API 200E chemiluminescent Nox Analyser
...,...,...,...,...,...,...
NL49564,Hoofddorp-Hoofdweg,"(52.327464,4.715008)",onbekend,onbekend,Chemiluminescentie,Teledyne API 200E chemiluminescent Nox Analyser
NL49565,Oude Meer-Aalsmeerderdijk,"(52.279991,4.770773)",regionaal,achtergrond,Chemiluminescentie,Teledyne API 200E chemiluminescent Nox Analyser
NL49701,Zaandam-Wagenschotpad,"(52.448011,4.816706)",stad,achtergrond,Chemiluminescentie,Thermo model 42w NO/Nox analyser
NL49703,Amsterdam-Spaarnwoude,"(52.398437,4.728581)",regionaal,achtergrond,Chemiluminescentie,Teledyne API 200E chemiluminescent Nox Analyser


### Clean meta data

In [24]:
# Seperate the latitude and longitude column and assign it to their own column
latitude_longitude = meta_data["Latitude,Longitude"].str.strip("()").str.split(",", n = 1, expand = True)

# Remove old Lat-long column
meta_data.drop("Latitude,Longitude", axis = 1, inplace=True)

latitude_longitude[[0,1]] = latitude_longitude[[0,1]].astype(float)

# Insert the chunk number column into the dataframe
meta_data.insert(1, column = "Latitude", value = latitude_longitude.iloc[:, 0])

# Insert the patient id column into the dataframe
meta_data.insert(2, column = "Longitude", value = latitude_longitude.iloc[:, 1])

In [25]:
meta_data

StationsCode,Stationsnaam,Latitude,Longitude,Stationsgebied,Stationstype,Meetprincipe,Meetopstelling
NL01485,Hoogvliet-Leemkuil,51.867411,4.355242,stad,achtergrond,Chemiluminescentie,Teledyne API 200E chemiluminescent Nox Analyser
NL01487,Rotterdam Zuid-Pleinweg,51.891147,4.480690,regionaal,verkeer,Chemiluminescentie,Teledyne API 200E chemiluminescent Nox Analyser
NL01488,Rotterdam Zuid-Zwartewaalstraat,51.893617,4.487528,stad,achtergrond,Chemiluminescentie,Teledyne API 200E chemiluminescent Nox Analyser
NL01489,Ridderkerk-Hogeweg,51.869431,4.580058,stad,verkeer,Chemiluminescentie,Teledyne API 200E chemiluminescent Nox Analyser
NL01491,Rotterdam-Oost Sidelinge A13,51.938472,4.430692,stad,verkeer,Chemiluminescentie,Teledyne API 200E chemiluminescent Nox Analyser
...,...,...,...,...,...,...,...
NL49564,Hoofddorp-Hoofdweg,52.327464,4.715008,onbekend,onbekend,Chemiluminescentie,Teledyne API 200E chemiluminescent Nox Analyser
NL49565,Oude Meer-Aalsmeerderdijk,52.279991,4.770773,regionaal,achtergrond,Chemiluminescentie,Teledyne API 200E chemiluminescent Nox Analyser
NL49701,Zaandam-Wagenschotpad,52.448011,4.816706,stad,achtergrond,Chemiluminescentie,Thermo model 42w NO/Nox analyser
NL49703,Amsterdam-Spaarnwoude,52.398437,4.728581,regionaal,achtergrond,Chemiluminescentie,Teledyne API 200E chemiluminescent Nox Analyser


In [14]:
test = pd.read_csv("test_file.csv", sep = ";")
test

,Component,Bep.periode,Eenheid,Begindatumtijd,Einddatumtijd,NL01485,NL01487,NL01488,NL01489,NL01491,...,NL49022,NL49546,NL49551,NL49553,NL49561,NL49564,NL49565,NL49701,NL49703,NL49704
0,NO2,uur,�g/m�,20210101 00:00,20210101 01:00,41.4,57.3,50.2,46.4,58.1,...,39.4,54.6,30.3,19.6,47.5,35.0,43.4,45.8,42.0,51.5
1,NO2,uur,�g/m�,20210101 01:00,20210101 02:00,41.8,51.4,47.8,43.0,56.2,...,36.0,53.4,39.7,17.9,49.2,33.6,44.4,40.2,43.3,48.9
2,NO2,uur,�g/m�,20210101 02:00,20210101 03:00,35.5,51.0,45.1,40.7,52.0,...,42.2,54.1,8.9,3.9,53.0,35.6,44.9,47.6,41.9,50.5
3,NO2,uur,�g/m�,20210101 03:00,20210101 04:00,20.4,52.9,49.9,45.3,54.6,...,41.2,55.1,9.0,3.0,56.6,38.9,56.0,42.7,39.3,47.1
4,NO2,uur,�g/m�,20210101 04:00,20210101 05:00,14.5,46.2,45.6,43.5,54.4,...,48.8,56.4,12.5,2.4,58.3,40.3,55.8,46.1,41.8,47.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
739,NO2,uur,�g/m�,20210131 19:00,20210131 20:00,37.1,31.3,30.4,21.8,27.5,...,21.2,18.5,18.8,16.3,19.4,24.6,14.9,12.9,18.1,17.9
740,NO2,uur,�g/m�,20210131 20:00,20210131 21:00,29.6,31.9,28.8,24.3,25.6,...,26.0,19.8,18.5,18.4,20.3,29.5,15.8,14.2,19.8,17.1
741,NO2,uur,�g/m�,20210131 21:00,20210131 22:00,28.5,27.1,25.6,22.2,24.4,...,20.9,18.8,18.2,20.0,18.5,28.1,15.2,14.6,19.5,18.2
742,NO2,uur,�g/m�,20210131 22:00,20210131 23:00,24.8,24.1,22.2,21.2,22.7,...,17.8,17.4,19.3,17.8,15.4,23.1,14.0,14.8,18.8,16.7


In [17]:
test_header = pd.read_csv("test_file_header.csv", sep = ";")
test_header

,Datum export,20210928 10:00,Unnamed: 2,Unnamed: 3,StationsCode,NL01485,NL01487,NL01488,NL01489,NL01491,...,NL49022,NL49546,NL49551,NL49553,NL49561,NL49564,NL49565,NL49701,NL49703,NL49704
0,Periode,20210101 00:00 - 20210201 00:00,NaN,NaN,Stationsnaam,Hoogvliet-Leemkuil,Rotterdam Zuid-Pleinweg,Rotterdam Zuid-Zwartewaalstraat,Ridderkerk-Hogeweg,Rotterdam-Oost Sidelinge A13,...,Amsterdam-Sportpark Ookmeer (Osdorp),Zaanstad-Hemkade,IJmuiden-Kanaaldijk,Wijk aan Zee-Burgemeester Rothestraat,Badhoevedorp-Sloterweg,Hoofddorp-Hoofdweg,Oude Meer-Aalsmeerderdijk,Zaandam-Wagenschotpad,Amsterdam-Spaarnwoude,Amsterdam-Hoogtij
1,Bron,https://data.rivm.nl/data/luchtmeetnet,NaN,NaN,"Latitude,Longitude","(51.867411,4.355242)","(51.891147,4.48069)","(51.893617,4.487528)","(51.869431,4.580058)","(51.938472,4.430692)",...,"(52.366811,4.793344)","(52.42023,4.83206)","(52.463039,4.601842)","(52.493992,4.601986)","(52.334003,4.774006)","(52.327464,4.715008)","(52.279991,4.770773)","(52.448011,4.816706)","(52.398437,4.728581)","(52.428017,4.773478)"
2,Beschrijving data,https://data.rivm.nl/data/luchtmeetnet/readme.pdf,NaN,NaN,Stationsgebied,stad,regionaal,stad,stad,stad,...,stad,stad,stad,stad,stad,onbekend,regionaal,stad,regionaal,stad
3,NaN,NaN,NaN,NaN,Stationstype,achtergrond,verkeer,achtergrond,verkeer,verkeer,...,achtergrond,industrie,industrie,industrie,achtergrond,onbekend,achtergrond,achtergrond,achtergrond,industrie
4,NaN,NaN,NaN,NaN,Meetprincipe,Chemiluminescentie,Chemiluminescentie,Chemiluminescentie,Chemiluminescentie,Chemiluminescentie,...,Chemiluminescentie,Chemiluminescentie,Chemiluminescentie,Chemiluminescentie,Chemiluminescentie,Chemiluminescentie,Chemiluminescentie,Chemiluminescentie,Chemiluminescentie,Chemiluminescentie
5,NaN,NaN,NaN,NaN,Meetopstelling,Teledyne API 200E chemiluminescent Nox Analyser,Teledyne API 200E chemiluminescent Nox Analyser,Teledyne API 200E chemiluminescent Nox Analyser,Teledyne API 200E chemiluminescent Nox Analyser,Teledyne API 200E chemiluminescent Nox Analyser,...,Thermo model 42w NO/Nox analyser,Teledyne API 200E chemiluminescent Nox Analyser,Teledyne API 200E chemiluminescent Nox Analyser,Teledyne API 200E chemiluminescent Nox Analyser,Teledyne API 200E chemiluminescent Nox Analyser,Teledyne API 200E chemiluminescent Nox Analyser,Teledyne API 200E chemiluminescent Nox Analyser,Thermo model 42w NO/Nox analyser,Teledyne API 200E chemiluminescent Nox Analyser,Teledyne API 200E chemiluminescent Nox Analyser


In [44]:
meta_data = test_header.iloc[:, 5:].T
meta_data.columns = test_header.StationsCode
meta_data

StationsCode,Stationsnaam,"Latitude,Longitude",Stationsgebied,Stationstype,Meetprincipe,Meetopstelling
NL01485,Hoogvliet-Leemkuil,"(51.867411,4.355242)",stad,achtergrond,Chemiluminescentie,Teledyne API 200E chemiluminescent Nox Analyser
NL01487,Rotterdam Zuid-Pleinweg,"(51.891147,4.48069)",regionaal,verkeer,Chemiluminescentie,Teledyne API 200E chemiluminescent Nox Analyser
NL01488,Rotterdam Zuid-Zwartewaalstraat,"(51.893617,4.487528)",stad,achtergrond,Chemiluminescentie,Teledyne API 200E chemiluminescent Nox Analyser
NL01489,Ridderkerk-Hogeweg,"(51.869431,4.580058)",stad,verkeer,Chemiluminescentie,Teledyne API 200E chemiluminescent Nox Analyser
NL01491,Rotterdam-Oost Sidelinge A13,"(51.938472,4.430692)",stad,verkeer,Chemiluminescentie,Teledyne API 200E chemiluminescent Nox Analyser
...,...,...,...,...,...,...
NL49564,Hoofddorp-Hoofdweg,"(52.327464,4.715008)",onbekend,onbekend,Chemiluminescentie,Teledyne API 200E chemiluminescent Nox Analyser
NL49565,Oude Meer-Aalsmeerderdijk,"(52.279991,4.770773)",regionaal,achtergrond,Chemiluminescentie,Teledyne API 200E chemiluminescent Nox Analyser
NL49701,Zaandam-Wagenschotpad,"(52.448011,4.816706)",stad,achtergrond,Chemiluminescentie,Thermo model 42w NO/Nox analyser
NL49703,Amsterdam-Spaarnwoude,"(52.398437,4.728581)",regionaal,achtergrond,Chemiluminescentie,Teledyne API 200E chemiluminescent Nox Analyser


In [45]:
# Seperate the latitude and longitude column and assign it to their own column
latitude_longitude = meta_data["Latitude,Longitude"].str.strip("()").str.split(",", n = 1, expand = True)

# Remove old Lat-long column
meta_data.drop("Latitude,Longitude", axis = 1, inplace=True)

latitude_longitude[[0,1]] = latitude_longitude[[0,1]].astype(float)

# Insert the chunk number column into the dataframe
meta_data.insert(1, column = "Latitude", value = latitude_longitude.iloc[:, 0])

# Insert the patient id column into the dataframe
meta_data.insert(2, column = "Longitude", value = latitude_longitude.iloc[:, 1])

In [46]:
meta_data

StationsCode,Stationsnaam,Latitude,Longitude,Stationsgebied,Stationstype,Meetprincipe,Meetopstelling
NL01485,Hoogvliet-Leemkuil,51.867411,4.355242,stad,achtergrond,Chemiluminescentie,Teledyne API 200E chemiluminescent Nox Analyser
NL01487,Rotterdam Zuid-Pleinweg,51.891147,4.480690,regionaal,verkeer,Chemiluminescentie,Teledyne API 200E chemiluminescent Nox Analyser
NL01488,Rotterdam Zuid-Zwartewaalstraat,51.893617,4.487528,stad,achtergrond,Chemiluminescentie,Teledyne API 200E chemiluminescent Nox Analyser
NL01489,Ridderkerk-Hogeweg,51.869431,4.580058,stad,verkeer,Chemiluminescentie,Teledyne API 200E chemiluminescent Nox Analyser
NL01491,Rotterdam-Oost Sidelinge A13,51.938472,4.430692,stad,verkeer,Chemiluminescentie,Teledyne API 200E chemiluminescent Nox Analyser
...,...,...,...,...,...,...,...
NL49564,Hoofddorp-Hoofdweg,52.327464,4.715008,onbekend,onbekend,Chemiluminescentie,Teledyne API 200E chemiluminescent Nox Analyser
NL49565,Oude Meer-Aalsmeerderdijk,52.279991,4.770773,regionaal,achtergrond,Chemiluminescentie,Teledyne API 200E chemiluminescent Nox Analyser
NL49701,Zaandam-Wagenschotpad,52.448011,4.816706,stad,achtergrond,Chemiluminescentie,Thermo model 42w NO/Nox analyser
NL49703,Amsterdam-Spaarnwoude,52.398437,4.728581,regionaal,achtergrond,Chemiluminescentie,Teledyne API 200E chemiluminescent Nox Analyser


In [81]:
pd.read_csv(Path(data_dir + "2021_01_NO2.csv"), nrows=6, sep=";", encoding='unicode_escape')

,Datum export,20210928 10:00,Unnamed: 2,Unnamed: 3,StationsCode,NL01485,NL01487,NL01488,NL01489,NL01491,...,NL49022,NL49546,NL49551,NL49553,NL49561,NL49564,NL49565,NL49701,NL49703,NL49704
0,Periode,20210101 00:00 - 20210201 00:00,NaN,NaN,Stationsnaam,Hoogvliet-Leemkuil,Rotterdam Zuid-Pleinweg,Rotterdam Zuid-Zwartewaalstraat,Ridderkerk-Hogeweg,Rotterdam-Oost Sidelinge A13,...,Amsterdam-Sportpark Ookmeer (Osdorp),Zaanstad-Hemkade,IJmuiden-Kanaaldijk,Wijk aan Zee-Burgemeester Rothestraat,Badhoevedorp-Sloterweg,Hoofddorp-Hoofdweg,Oude Meer-Aalsmeerderdijk,Zaandam-Wagenschotpad,Amsterdam-Spaarnwoude,Amsterdam-Hoogtij
1,Bron,https://data.rivm.nl/data/luchtmeetnet,NaN,NaN,"Latitude,Longitude","(51.867411,4.355242)","(51.891147,4.48069)","(51.893617,4.487528)","(51.869431,4.580058)","(51.938472,4.430692)",...,"(52.366811,4.793344)","(52.42023,4.83206)","(52.463039,4.601842)","(52.493992,4.601986)","(52.334003,4.774006)","(52.327464,4.715008)","(52.279991,4.770773)","(52.448011,4.816706)","(52.398437,4.728581)","(52.428017,4.773478)"
2,Beschrijving data,https://data.rivm.nl/data/luchtmeetnet/readme.pdf,NaN,NaN,Stationsgebied,stad,regionaal,stad,stad,stad,...,stad,stad,stad,stad,stad,onbekend,regionaal,stad,regionaal,stad
3,NaN,NaN,NaN,NaN,Stationstype,achtergrond,verkeer,achtergrond,verkeer,verkeer,...,achtergrond,industrie,industrie,industrie,achtergrond,onbekend,achtergrond,achtergrond,achtergrond,industrie
4,NaN,NaN,NaN,NaN,Meetprincipe,Chemiluminescentie,Chemiluminescentie,Chemiluminescentie,Chemiluminescentie,Chemiluminescentie,...,Chemiluminescentie,Chemiluminescentie,Chemiluminescentie,Chemiluminescentie,Chemiluminescentie,Chemiluminescentie,Chemiluminescentie,Chemiluminescentie,Chemiluminescentie,Chemiluminescentie
5,NaN,NaN,NaN,NaN,Meetopstelling,Teledyne API 200E chemiluminescent Nox Analyser,Teledyne API 200E chemiluminescent Nox Analyser,Teledyne API 200E chemiluminescent Nox Analyser,Teledyne API 200E chemiluminescent Nox Analyser,Teledyne API 200E chemiluminescent Nox Analyser,...,Thermo model 42w NO/Nox analyser,Teledyne API 200E chemiluminescent Nox Analyser,Teledyne API 200E chemiluminescent Nox Analyser,Teledyne API 200E chemiluminescent Nox Analyser,Teledyne API 200E chemiluminescent Nox Analyser,Teledyne API 200E chemiluminescent Nox Analyser,Teledyne API 200E chemiluminescent Nox Analyser,Thermo model 42w NO/Nox analyser,Teledyne API 200E chemiluminescent Nox Analyser,Teledyne API 200E chemiluminescent Nox Analyser


In [4]:
pd.read_csv(Path(data_dir + "2020_NO2.csv"), nrows=6, sep=";", encoding='unicode_escape')

,Datum export,20210316 13:33,Unnamed: 2,Unnamed: 3,StationsCode,NL01485,NL01487,NL01488,NL01489,NL01491,...,NL49022,NL49546,NL49551,NL49553,NL49561,NL49564,NL49565,NL49701,NL49703,NL49704
0,Periode,20200101 00:00 - 20210101 00:00,NaN,NaN,Stationsnaam,Hoogvliet-Leemkuil,Rotterdam Zuid-Pleinweg,Rotterdam Zuid-Zwartewaalstraat,Ridderkerk-Hogeweg,Rotterdam-Oost Sidelinge A13,...,Amsterdam-Sportpark Ookmeer (Osdorp),Zaanstad-Hemkade,IJmuiden-Kanaaldijk,Wijk aan Zee-Burgemeester Rothestraat,Badhoevedorp-Sloterweg,Hoofddorp-Hoofdweg,Oude Meer-Aalsmeerderdijk,Zaandam-Wagenschotpad,Amsterdam-Spaarnwoude,Amsterdam-Hoogtij
1,Bron,https://data.rivm.nl/data/luchtmeetnet,NaN,NaN,"Latitude,Longitude","(51.867411,4.355242)","(51.891147,4.48069)","(51.893617,4.487528)","(51.869431,4.580058)","(51.938472,4.430692)",...,"(52.366811,4.793344)","(52.42023,4.83206)","(52.463039,4.601842)","(52.493992,4.601986)","(52.334003,4.774006)","(52.327464,4.715008)","(52.279991,4.770773)","(52.448011,4.816706)","(52.398437,4.728581)","(52.428017,4.773478)"
2,Beschrijving data,https://data.rivm.nl/data/luchtmeetnet/readme.pdf,NaN,NaN,Stationsgebied,stad,regionaal,stad,stad,stad,...,stad,stad,stad,stad,stad,onbekend,regionaal,stad,regionaal,stad
3,NaN,NaN,NaN,NaN,Stationstype,achtergrond,verkeer,achtergrond,verkeer,verkeer,...,achtergrond,industrie,industrie,industrie,achtergrond,onbekend,achtergrond,achtergrond,achtergrond,industrie
4,NaN,NaN,NaN,NaN,Meetprincipe,Chemiluminescentie,Chemiluminescentie,Chemiluminescentie,Chemiluminescentie,Chemiluminescentie,...,Chemiluminescentie,Chemiluminescentie,Chemiluminescentie,Chemiluminescentie,Chemiluminescentie,Chemiluminescentie,Chemiluminescentie,Chemiluminescentie,Chemiluminescentie,Chemiluminescentie
5,NaN,NaN,NaN,NaN,Meetopstelling,Teledyne API 200E chemiluminescent Nox Analyser,Teledyne API 200E chemiluminescent Nox Analyser,Teledyne API 200E chemiluminescent Nox Analyser,Teledyne API 200E chemiluminescent Nox Analyser,Teledyne API 200E chemiluminescent Nox Analyser,...,Thermo model 42w NO/Nox analyser,Teledyne API 200E chemiluminescent Nox Analyser,Teledyne API 200E chemiluminescent Nox Analyser,Teledyne API 200E chemiluminescent Nox Analyser,Teledyne API 200E chemiluminescent Nox Analyser,Teledyne API 200E chemiluminescent Nox Analyser,Teledyne API 200E chemiluminescent Nox Analyser,Thermo model 42w NO/Nox analyser,Teledyne API 200E chemiluminescent Nox Analyser,Teledyne API 200E chemiluminescent Nox Analyser


# Test wide to long

In [47]:
data_url = "https://goo.gl/ioc2Td"
gapminder = pd.read_csv(data_url)
print(gapminder.head(3))

  continent  country  gdpPercap_1952  gdpPercap_1957  gdpPercap_1962  \
0    Africa  Algeria     2449.008185     3013.976023     2550.816880   
1    Africa   Angola     3520.610273     3827.940465     4269.276742   
2    Africa    Benin     1062.752200      959.601080      949.499064   

   gdpPercap_1967  gdpPercap_1972  gdpPercap_1977  gdpPercap_1982  \
0     3246.991771     4182.663766     4910.416756     5745.160213   
1     5522.776375     5473.288005     3008.647355     2756.953672   
2     1035.831411     1085.796879     1029.161251     1277.897616   

   gdpPercap_1987  ...    pop_1962    pop_1967    pop_1972    pop_1977  \
0     5681.358539  ...  11000948.0  12760499.0  14760787.0  17152804.0   
1     2430.208311  ...   4826015.0   5247469.0   5894858.0   6162675.0   
2     1225.856010  ...   2151895.0   2427334.0   2761407.0   3168267.0   

     pop_1982    pop_1987    pop_1992    pop_1997  pop_2002  pop_2007  
0  20033753.0  23254956.0  26298373.0  29072015.0  31287142  3333

In [48]:
lifeExp = gapminder.loc[:, gapminder.columns.str.contains('^life|^c')]
print(lifeExp.head(n=3))

  continent  country  lifeExp_1952  lifeExp_1957  lifeExp_1962  lifeExp_1967  \
0    Africa  Algeria        43.077        45.685        48.303        51.407   
1    Africa   Angola        30.015        31.999        34.000        35.985   
2    Africa    Benin        38.223        40.358        42.618        44.885   

   lifeExp_1972  lifeExp_1977  lifeExp_1982  lifeExp_1987  lifeExp_1992  \
0        54.518        58.014        61.368        65.799        67.744   
1        37.928        39.483        39.942        39.906        40.647   
2        47.014        49.190        50.904        52.337        53.919   

   lifeExp_1997  lifeExp_2002  lifeExp_2007  
0        69.152        70.994        72.301  
1        40.963        41.003        42.731  
2        54.777        54.406        56.728  


In [50]:
gapminder_tidy = lifeExp.melt(id_vars=["continent", "country"], 
                              var_name="year", 
                              value_name="lifeExp")
gapminder_tidy.head(n=10)

,continent,country,year,lifeExp
0,Africa,Algeria,lifeExp_1952,43.077
1,Africa,Angola,lifeExp_1952,30.015
2,Africa,Benin,lifeExp_1952,38.223
3,Africa,Botswana,lifeExp_1952,47.622
4,Africa,Burkina Faso,lifeExp_1952,31.975
5,Africa,Burundi,lifeExp_1952,39.031
6,Africa,Cameroon,lifeExp_1952,38.523
7,Africa,Central African Republic,lifeExp_1952,35.463
8,Africa,Chad,lifeExp_1952,38.092
9,Africa,Comoros,lifeExp_1952,40.715
